In [ ]:
#!pip uninstall tensorflow
#!pip install torch torchvision -U

!pip uninstall torch torchvision
!pip install torch==2.2.0 torchvision==0.17.0
#!pip uninstall tensorflow
#!pip uninstall jax

Found existing installation: torch 2.2.0+cpu
Uninstalling torch-2.2.0+cpu:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/torch-2.2.0+cpu.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? y
  Successfully uninstalled torch-2.2.0+cpu
Found existing installation: torchvision 0.17.0+cpu
Uninstalling torchvision-0.17.0+cpu:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/torchvision-0.17.0+cpu.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libjpeg.ceea7512.so.62
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libpng16.7f72a3c5.so.16
    /usr/local/lib/python3.10/dist-packages/torchvision.libs/libz.37eba27a.so.1
    /usr/local/lib/python3.10/dist-packages/torchvision/*
Proceed (Y/

In [ ]:
from importlib.metadata import version
version("jax")

'0.4.26'

In [ ]:
!git clone https://github.com/Alberknyis/bad-teaching-unlearning.git

fatal: destination path 'bad-teaching-unlearning' already exists and is not an empty directory.


In [ ]:
import sys
sys.path.insert(0,'/content/bad-teaching-unlearning')

In [ ]:
import torch

from dataset import *
from model import ResNet18
from unlearn import *
from metrics import UnLearningScore
from utils import *
from torch.utils.data import DataLoader

In [ ]:
#import torch_xla
import torch_xla.core.xla_model as xm
device = xm.xla_device()

In [ ]:
train_ds = CustomCIFAR100(root='.', train=True,download=True, transform=transform_train)
valid_ds = CustomCIFAR100(root='.', train=False,download=True, transform=transform_train)

batch_size = 256
#batch_size = 128
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=32, pin_memory=True)
valid_dl = DataLoader(valid_ds, batch_size, num_workers=32, pin_memory=True)

#train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=8, pin_memory=True)
#valid_dl = DataLoader(valid_ds, batch_size, num_workers=8, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
num_classes = 100
classwise_train = {}
for i in range(num_classes):
    classwise_train[i] = []

for img, label, clabel in train_ds:
    classwise_train[label].append((img, label, clabel))

classwise_test = {}
for i in range(num_classes):
    classwise_test[i] = []

for img, label, clabel in valid_ds:
    classwise_test[label].append((img, label, clabel))

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# train the model
#device = 'cuda'
model = ResNet18(num_classes = 20, pretrained = True).to(device)
epochs = 5
#epochs = 1
history = fit_one_cycle(epochs, model, train_dl, valid_dl, device = device)
torch.save(model.state_dict(), "ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt")

Epoch [0], last_lr: 0.00100, train_loss: 1.2409, val_loss: 1.0527, val_acc: 66.9727
Epoch [1], last_lr: 0.00100, train_loss: 0.7352, val_loss: 0.8059, val_acc: 75.4199
Epoch [2], last_lr: 0.00100, train_loss: 0.5268, val_loss: 0.7724, val_acc: 77.9785
Epoch [3], last_lr: 0.00100, train_loss: 0.4539, val_loss: 0.9825, val_acc: 72.2656
Epoch [4], last_lr: 0.00100, train_loss: 0.3325, val_loss: 0.8121, val_acc: 77.5195


In [ ]:
torch.save(model.state_dict(), "ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt")

In [ ]:
# load the trained model
#device = 'cuda'
model = ResNet18(num_classes = 20, pretrained = True).to(device)
model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location='cuda'))

<All keys matched successfully>

# Forgetting Rocket
The Rocket is class 69 in CIFAR100 and belongs to Super Class 19 (Vehicles) in CIFAR Super 20.

In [ ]:
# Getting the forget and retain validation data
forget_valid = []
forget_classes = [69]
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            forget_valid.append((img, label, clabel))

retain_valid = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            retain_valid.append((img, label, clabel))

forget_train = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            forget_train.append((img, label, clabel))

retain_train = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            retain_train.append((img, label, clabel))

forget_valid_dl = DataLoader(forget_valid, batch_size, num_workers=32, pin_memory=True)

retain_valid_dl = DataLoader(retain_valid, batch_size, num_workers=32, pin_memory=True)

forget_train_dl = DataLoader(forget_train, batch_size, num_workers=32, pin_memory=True)
retain_train_dl = DataLoader(retain_train, batch_size, num_workers=32, pin_memory=True, shuffle = True)
import random
retain_train_subset = random.sample(retain_train, int(0.3*len(retain_train)))
retain_train_subset_dl = DataLoader(retain_train_subset, batch_size, num_workers=32, pin_memory=True, shuffle = True)

In [ ]:
# Performance of Fully trained model on retain set
evaluate(model, retain_valid_dl, device)

{'Loss': 0.8249897956848145, 'Acc': 77.42457580566406}

In [ ]:
# Performance of Fully trained model on retain set
evaluate(model, forget_valid_dl, device)

{'Loss': 0.8302143812179565, 'Acc': 73.0}

## Retrain the model from Scratch
Create Retrained Model (Gold model). This is the model trained from scratch without forget data.

In [ ]:
#device = 'cuda'
gold_model = ResNet18(num_classes = 20, pretrained = True).to(device)
epochs = 5
#epochs = 1
history = fit_one_cycle(epochs, gold_model, retain_train_dl, retain_valid_dl, device = device)
torch.save(gold_model.state_dict(), "ResNET18_CIFAR100Super20_Pretrained_Gold_Class69_5_Epochs.pt")

Epoch [0], last_lr: 0.00100, train_loss: 1.2551, val_loss: 1.0415, val_acc: 68.2026
Epoch [1], last_lr: 0.00100, train_loss: 0.7258, val_loss: 0.9929, val_acc: 73.2682
Epoch [2], last_lr: 0.00100, train_loss: 0.5329, val_loss: 1.8756, val_acc: 63.1897
Epoch [3], last_lr: 0.00100, train_loss: 0.4042, val_loss: 0.9015, val_acc: 75.5180
Epoch [4], last_lr: 0.00100, train_loss: 0.2898, val_loss: 0.9702, val_acc: 76.2497


In [ ]:
#device = 'cuda'
gold_model = ResNet18(num_classes = 20, pretrained = True).to(device)
gold_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_Gold_Class69_5_Epochs.pt", map_location=device))

<All keys matched successfully>

In [ ]:
# evaluate gold model on forget set
evaluate(gold_model, forget_valid_dl, device)

{'Loss': 8.612824440002441, 'Acc': 1.0}

In [ ]:
# evaluate gold model on retain set
evaluate(gold_model, retain_valid_dl, device)

{'Loss': 0.9701842665672302, 'Acc': 76.24967193603516}

## UnLearning via proposed method

In [ ]:
#device = 'cuda'
unlearning_teacher = ResNet18(num_classes = 20, pretrained = False).to(device).eval()
student_model = ResNet18(num_classes = 20, pretrained = False).to(device)
student_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = device))
model = model.eval()

KL_temperature = 1

optimizer = torch.optim.Adam(student_model.parameters(), lr = 0.0001)

blindspot_unlearner(model = student_model, unlearning_teacher = unlearning_teacher, full_trained_teacher = model,
          retain_data = retain_train_subset, forget_data = forget_train, epochs = 1, optimizer = optimizer, lr = 0.0001,
          batch_size = 256, num_workers = 32, device = device, KL_temperature = KL_temperature)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2949: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:266: UserWarning: aten::kl_div: an autograd kernel was not registered to the Autograd key(s) but we are trying to backprop through it. This may lead to silently incorrect behavior. This behavior is deprecated and will be removed in a future version of PyTorch. If your operator is differen

Epoch 1 Unlearning Loss 0.0056249951012432575


In [ ]:
# performance of unlearned model on forget set
evaluate(student_model, forget_valid_dl, device)

{'Loss': 3.3927807807922363, 'Acc': 1.0}

In [ ]:
# performance of unlearned model on retain set
evaluate(student_model, retain_valid_dl, device)

{'Loss': 0.7348727583885193, 'Acc': 78.84661865234375}

### Measure ZRF (Unlearning Score)

In [ ]:
#print("Initial Score: {}".format(UnLearningScore(model, unlearning_teacher, forget_valid_dl, 256, 'cuda')))
#print("Our Score: {}".format(UnLearningScore(student_model, unlearning_teacher, forget_valid_dl, 256, 'cuda')))
#print("Gold Score: {}".format(UnLearningScore(gold_model, unlearning_teacher, forget_valid_dl, 256, 'cuda')))
#print("JS Div: {}".format(1-UnLearningScore(gold_model, student_model, forget_valid_dl, 256, 'cuda')))
print("Initial Score: {}".format(UnLearningScore(model, unlearning_teacher, forget_valid_dl, 256, device)))
print("Our Score: {}".format(UnLearningScore(student_model, unlearning_teacher, forget_valid_dl, 256, device)))
print("Gold Score: {}".format(UnLearningScore(gold_model, unlearning_teacher, forget_valid_dl, 256, device)))
print("JS Div: {}".format(1-UnLearningScore(gold_model, student_model, forget_valid_dl, 256, device)))

Initial Score: 0.9227719306945801
Our Score: 0.989225447177887
Gold Score: 0.9363023638725281
JS Div: 0.04250079393386841


## Unlearning using Amnesiac unlearning

In [ ]:
unlearninglabels = list(range(20))
unlearninglabels.remove(19)
unlearning_train_set = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            unlearning_train_set.append((img, label, random.choice(unlearninglabels)))



for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            unlearning_train_set.append((img, label, clabel))

In [ ]:
unlearning_train_set_dl = DataLoader(unlearning_train_set, batch_size, num_workers = 32, pin_memory = True, shuffle = True)

In [ ]:
#device = 'cuda'
student_model = ResNet18(num_classes = 20, pretrained = True).to(device)
student_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = 'cuda'))
epochs = 3

history = fit_one_unlearning_cycle(epochs, student_model, unlearning_train_set_dl, retain_valid_dl, device = device, lr = 0.0001)

Epoch [0], last_lr: 0.00010, train_loss: 0.1328, val_loss: 0.6601, val_acc: 84.3631
Epoch [1], last_lr: 0.00010, train_loss: 0.0564, val_loss: 0.7640, val_acc: 84.3790
Epoch [2], last_lr: 0.00010, train_loss: 0.0386, val_loss: 0.8184, val_acc: 84.4430


In [ ]:
#this is where the allowance ran out

In [ ]:
print("Forget Performance: {}".format(evaluate(student_model, forget_valid_dl, device)))
print("Retain Performance: {}".format(evaluate(student_model, retain_valid_dl, device)))

Forget Performance: {'Loss': 3.6175918579101562, 'Acc': 4.0}
Retain Performance: {'Loss': 0.5542724132537842, 'Acc': 84.37849426269531}


## Unlearning using UNSIR (Class 0)

In [ ]:
num_classes = 20
classwise_train = {}
for i in range(num_classes):
    classwise_train[i] = []

for img, label, clabel in train_ds:
    classwise_train[clabel].append((img, label, clabel))

classwise_test = {}
for i in range(num_classes):
    classwise_test[i] = []

for img, label, clabel in valid_ds:
    classwise_test[clabel].append((img, label, clabel))

In [ ]:
# Getting the forget and retain validation data
forget_valid = []
forget_classes = [0]
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            forget_valid.append((img, label, clabel))

retain_valid = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_test[cls]:
            retain_valid.append((img, label, clabel))

forget_train = []
for cls in range(num_classes):
    if cls in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            forget_train.append((img, label, clabel))

retain_train = []
for cls in range(num_classes):
    if cls not in forget_classes:
        for img, label, clabel in classwise_train[cls]:
            retain_train.append((img, label, clabel))

forget_valid_dl = DataLoader(forget_valid, batch_size, num_workers=32, pin_memory=True)

retain_valid_dl = DataLoader(retain_valid, batch_size, num_workers=32, pin_memory=True)

forget_train_dl = DataLoader(forget_train, batch_size, num_workers=32, pin_memory=True)
retain_train_dl = DataLoader(retain_train, batch_size, num_workers=32, pin_memory=True, shuffle = True)

In [ ]:
#collect some samples from each class
num_samples = 500
retain_samples = []
for i in range(num_classes):
    if i not in forget_classes:
        retain_samples += classwise_train[i][:num_samples]

In [ ]:
noise_batch_size = 256

In [ ]:
#device = 'cuda'
student_model = ResNet18(num_classes = 20, pretrained = False).to(device)
student_model.load_state_dict(torch.load("ResNET18_CIFAR100Super20_Pretrained_ALL_CLASSES_5_Epochs.pt", map_location = device))

<All keys matched successfully>

In [ ]:
noise = UNSIR_noise(batch_size, 3, 224, 224).to(device)

In [ ]:
forget_class_label = 0
#num_epochs = 250
num_epochs = 1

In [ ]:
noise =  UNSIR_noise_train(noise, student_model, forget_class_label, num_epochs,\
                           noise_batch_size, device=device)

Loss: 751.0631713867188


In [ ]:
#takes 19 years
noisy_loader = UNSIR_create_noisy_loader(noise, forget_class_label\
                                         , retain_samples, batch_size, device=device)

In [ ]:
#impair step
epochs = 1
history = fit_one_unlearning_cycle(epochs, student_model, noisy_loader, retain_valid_dl, device = device, lr = 0.0001)

Epoch [0], last_lr: 0.00010, train_loss: 0.7036, val_loss: 0.7013, val_acc: 79.1133


In [ ]:
print("Forget Performance: {}".format(evaluate(student_model, forget_valid_dl, device)))
print("Retain Performance: {}".format(evaluate(student_model, retain_valid_dl, device)))

In [ ]:
#repair step
other_samples = []
for i in range(len(retain_samples)):
    other_samples.append((retain_samples[i][0].cpu(), torch.tensor(retain_samples[i][2]),
                            torch.tensor(retain_samples[i][2])))

heal_loader = torch.utils.data.DataLoader(other_samples, batch_size=batch_size, shuffle = True)
epochs = 1
history = fit_one_unlearning_cycle(epochs, student_model, heal_loader, retain_valid_dl, device = device, lr = 0.0001)

Epoch [0], last_lr: 0.00010, train_loss: 0.0178, val_loss: 0.6395, val_acc: 84.6217


In [ ]:
print("Forget Performance: {}".format(evaluate(student_model, forget_valid_dl, device)))
print("Retain Performance: {}".format(evaluate(student_model, retain_valid_dl, device)))

Forget Performance: {'Loss': 4.577937602996826, 'Acc': 17.43404197692871}
Retain Performance: {'Loss': 0.639464259147644, 'Acc': 84.62171173095703}
